In [1]:
from coppeliasim_zmqremoteapi_client  import RemoteAPIClient
import numpy as np
import time
import math

In [2]:
client = RemoteAPIClient()
sim = client.require('sim')

In [3]:
sim.setStepping(True)

sim.startSimulation()
while (t := sim.getSimulationTime()) < 3:
    print(f'Simulation time: {t:.2f} [s]')
    sim.step()
sim.stopSimulation()

In [4]:
# Joints
LEGS = []

j1=sim.getObject('./S11')
j2=sim.getObject('./S12')

LEGS.append((j1, j2))



j1=sim.getObject('./S21')
j2=sim.getObject('./S22')

LEGS.append((j1, j2))



j1=sim.getObject('./S31')
j2=sim.getObject('./S32')

LEGS.append((j1, j2))



j1=sim.getObject('./S41')
j2=sim.getObject('./S42')

LEGS.append((j1, j2))



LEGS=np.array(LEGS)

print(LEGS)

[[28 30]
 [16 18]
 [24 26]
 [20 22]]


In [5]:
# Position by state mapping


Y_PLANE = -1/3
X_PLANE = 1


Y_UP = 1

position_states = {
    #basic positions
    "start":            ( 0, -1),
    "sit"  :            ( 0,  0),

    #short stand positions
    "center" :          ( 0*X_PLANE,  Y_PLANE),
    "front":            ( 1/4*X_PLANE,Y_PLANE),
    "back" :            (-1/4*X_PLANE,Y_PLANE),

    #long stand positions
    "long_front" :      ( 1/2*X_PLANE, Y_PLANE),
    "long_back" :       (-1/2*X_PLANE, Y_PLANE),




    #short stand positions
    "center_down" :          ( 0*X_PLANE,  Y_PLANE -0.1),
    "front_down":            ( 1/4*X_PLANE,Y_PLANE -0.1),
    "back_down" :            (-1/4*X_PLANE,Y_PLANE -0.1),

    #long stand positions
    "long_front_down" :      ( 1/2*X_PLANE, Y_PLANE -0.1),
    "long_back_down" :       (-1/2*X_PLANE, Y_PLANE -0.1),



    #short stand positions
    "center_up" :          ( 0*X_PLANE,  Y_PLANE +0.1),
    "front_up":            ( 1/4*X_PLANE,Y_PLANE +0.1),
    "back_up" :            (-1/4*X_PLANE,Y_PLANE +0.1),

    #long stand positions
    "long_front_up" :      ( 1/2*X_PLANE, Y_PLANE  +0.1),
    "long_back_up" :       (-1/2*X_PLANE, Y_PLANE  +0.1),


    "raise_1" : (0.5*X_PLANE,Y_UP),
    "raise_2" : ( 1.0*X_PLANE,Y_UP+0.5)
}

forwards_states = {
    "front"      : "long_front",
    "center"     : "front",   
    "back"       : "center",
    "long_back"  : "back",

    "front_up"      : "long_front_up",
    "center_up"     : "front_up",   
    "back_up"       : "center_up",
    "long_back_up"  : "back_up",

    "front_down"      : "long_front_down",
    "center_down"     : "front_down",   
    "back_down"       : "center_down",
    "long_back_down"  : "back_down",    
}

backwards_states = {
    "long_front" : "front",
    "front"      : "center",
    "center"     : "back",   
    "back"       : "long_back",

    "long_front_up" : "front_up",
    "front_up"      : "center_up",
    "center_up"     : "back_up",   
    "back_up"       : "long_back_up",

    "long_front_down" : "front_down",
    "front_down"      : "center_down",
    "center_down"     : "back_down",   
    "back_down"       : "long_back_down",
}

def position(state):
    if isinstance(state, list):
        return [position(substate) for substate in state]

    assert position_states.get(state) is not None,\
        f"State {state} has no position assigned"
    
    return position_states[state]

def forwards(old_state):
    if isinstance(old_state, list):
        return [backwards(old_substate) for old_substate in old_state]

    assert forwards_states.get(old_state) is not None,\
        f"State {old_state} has no position forwards state assigned"
    
    return forwards_states[old_state]

def backwards(old_state):
    if isinstance(old_state, list):
        return [forwards(old_substate) for old_substate in old_state]

    assert backwards_states.get(old_state) is not None,\
        f"State {old_state} has no position backwards state assigned"
    
    return backwards_states[old_state]

def upwards(states):
    return ["center" for _ in states]

def downwards(states):
    return ["sit" for _ in states]

In [6]:
# Define the walk states

walk_states = {
    "start" : [
        ('u', 'A', 'l'),
        ('f', 0, 'p'),('f', 'A', 'l'),('f', 3, 'p'),
    ],
    "cycle" :  [
        ('ff', 1, 'p'),('f', 'A', 'l'),('ff', 2, 'p'),
        ('ff', 0, 'p'),('f', 'A', 'l'),('ff', 3, 'p'),
    ],
    "end" : [
        ('f', 1, 'p'), ('f', 'A', 'l'),('f', 2, 'p'),
        ('b', 'A', 'l'),
        ('d', 'A', 'l'),
    ]
}

rot_states = {
    "start" : [
        ('u', 'A', 'l'),
        ("center_up", [0,1,2,3], 'l'),("center_down", [0], 'l')
    ],
    "cycle" : [
        ('f', 0, 'p'), 
        ('f', 'A', 'l'),
        ('b', 0, 'l'),
        ('b', 'A', 'l'),
    ],
    "end" : [
        ('d', 'A', 'l')
    ]
}

maneta_states = {
    "start" :     [
        ('u', 'A', 'l'),
        ('center_up', [2,3], 'l'),('center_down', [0,1], 'l'),
        ('back_down', [0], 'p'),('back_down', [1], 'p'),
        ('front_up', [2,3], 'l'),      
        ('raise_1', [0], 'l'),('long_front_up', [3], 'l')
    ],
    "cycle" : [
        ('raise_2', [0], 'p'),('raise_1', [0], 'p')
    ],
    "end" : [
        ('center_up', [0], 'l'),
        ('d', 'A', 'l')
    ]
}

sit_states = {
    "start" : [    
        ('u', 'A', 'l'),
        ('center_up', [2,3], 'l'),('center_down', [0,1], 'l'),
        
        ('back_down', [0], 'p'),('back_down', [1], 'p'),

        ('front_up', [2,3], 'l'),
    ],
    "cycle" : [
        ('back_down', [0], 'l'),
        ('back_down', [0], 'l'),
        ('back_down', [0], 'l'),
        ('back_down', [0], 'l'),
        ('back_down', [0], 'l')
    ],
    "end" : [
        ('d', 'A', 'l')
    ]
}

walk_back_states = {
    "start" : [
        ('u', 'A', 'l'),
        ('f', 'A', 'l'),
        ('b', 3, 'p'),('b', 'A', 'l'),('b', 0, 'p'),
    ],
    "cycle" : [
        ('bb', 2, 'p'),('b', 'A', 'l'),('bb', 1, 'p'),
        ('bb', 3, 'p'),('b', 'A', 'l'),('bb', 0, 'p'),
    ],
    "end" : [
        ('b', 2, 'p'),('b', 'A', 'l'),('b', 2, 'p'),
        ('u', 'A', 'l'),
    ]
}

In [7]:
# Foot space interpretation

def foot_to_polar(foot):
    (x,y) = foot

    Y = (1-y)/2
    #X = x * np.sqrt(1-Y**2)
    X = x * np.sqrt(5)/3
    
    R = np.sqrt(X**2 + Y**2)
    if R > +1: R = +1
    if R < -1: R = -1
    assert R <= 1, f"Radius out of bounds: {R}"


    A = np.arctan2(Y, X)
    assert -np.pi <= A <= np.pi, f"Angle out of bounds: {A*180/np.pi}"


    (R, A) = (round(R, 4), round(np.pi/2 - A, 4))
    #print(f"Calculated polar: radius={R}º, angle={A*180/np.pi}º")
    return (R, A)

def isPossible(foot):
    try:
        foot_to_polar(foot)
    except:
        return False
    
    return True

def polar_to_position(polar):
    (r,a) = polar
    th1 = a - np.arccos(r)
    th2 = 2*np.arccos(r)

    th1 = th1 - round(th1/(np.pi*2))*2*np.pi
    th2 = th2 - round(th2/(np.pi*2))*2*np.pi

    th1=round(th1*180/np.pi,4)
    th2=round(th2*180/np.pi,4)
    #print(f"Calculated angles: theta1={th1}º, theta2={th2}º")

    return (th1, th2)

def conversion(foot, angle_type='radians'):
    polar = foot_to_polar(foot)
    position = polar_to_position(polar)
    if angle_type == 'radians': position = (np.deg2rad(position[0]), np.deg2rad(position[1]))
    return position

isPossible((0,0))

True

In [8]:
# Foot space interpolations

def linear_interpolation(old, new, factor):
    (old1, old2) = old
    (new1, new2) = new

    now1 = old1 + factor * (new1 - old1)
    now2 = old2 + factor * (new2 - old2)

    now = (round(now1, 4), round(now2, 4))

    assert isPossible(now), f"ERROR, impossible position {now}"
    return now

def square_interpolation(old, new, factor, height=1/6):
    linear_length = np.sqrt((new[0] - old[0])**2 + (new[1] - old[1])**2)

    total_length = linear_length + height*2
    end_first_tram = height / total_length
    end_second_tram = (height + linear_length) / total_length
    end_third_tram = 1

    (x_o, y_o) = old
    (x_n, y_n) = new
    now = None

    if factor < end_first_tram:
        actual_factor = factor / end_first_tram
        
        start = (x_o, y_o)
        end = (x_o, y_o+height)
        now = linear_interpolation(start, end, actual_factor)
    
    elif factor < end_second_tram:
        actual_factor = (factor-end_first_tram) / (end_second_tram-end_first_tram)
        
        start = (x_o, y_o+height)
        end = (x_n, y_n+height)
        now = linear_interpolation(start, end, actual_factor)
    
    else:
        actual_factor = (factor-end_second_tram) / (end_third_tram-end_second_tram)
        
        start = (x_n, y_n+height)
        end = (x_n, y_n)
        now = linear_interpolation(start, end, actual_factor)
    
    assert isPossible(now), f"ERROR, impossible position {now}"
    return now


def parabolic_interpolation(old, new, factor, height=1/6):
    (old1, old2) = old
    (new1, new2) = new

    height_factor = 4 * factor * (1 - factor)

    now1 = old1 + factor * (new1 - old1)
    now2 = old2 + factor * (new2 - old2) + height * height_factor

    now = (round(now1, 4), round(now2, 4))

    assert isPossible(now), f"ERROR, impossible position {now}"
    return now

def interpolation(old, new, factor, method='linear', height=1/6):
    if method == 'linear':
        return linear_interpolation(old, new, factor)
    elif method == 'parabolic':
        return parabolic_interpolation(old, new, factor, height)
    elif method == 'square':
        return square_interpolation(old, new, factor, height)
    else:
        raise ValueError("Unknown interpolation method: {}".format(method))

In [9]:
# Set position by foot positions
# !!! add threads

def check_position(leg, now):
    (up_pred, down_pred) = conversion(now)
    (joint1, joint2) = leg

    up_pred = round(up_pred, 1)
    down_pred = round(down_pred, 1)

    up_true = round(sim.getJointPosition(int(joint1)), 1)
    down_true = round(sim.getJointPosition(int(joint2)), 1) 
    
    assert up_true == up_pred, f"ERROR, joint1 position {up_true} does not match predicted {up_pred}"
    assert down_true == down_pred, f"ERROR, joint2 position {down_true} does not match predicted {down_pred}"

def set_position(leg, now):
    (joint_up, joint_down) = leg
    (pos_up, pos_down) = conversion(now)

    sim.setJointTargetPosition(int(joint_up), pos_up)
    sim.setJointTargetPosition(int(joint_down), pos_down)

    #check_position(leg, now)


def set_legs(legs, olds, news, steps=10, duration=1, inter_method='linear'):

    for i in range(steps):
        tic = time.time()

        factor = (i+1)/steps

        for leg, old, new in zip(legs, olds, news):
            #get now postition
            now = interpolation(old, new, factor, method=inter_method, height=2/6)

            #set position
            set_position(leg, now)

        toc = time.time()
        dt = toc-tic

        if dt < duration/steps:
            time.sleep(duration/steps - dt)
        else:
            print(f"Warning: Step took too long: {dt:.3f} seconds (expected less than {duration/steps:.3f})")


def init_sim(legs):
    for leg in legs:
        (joint1, joint2) = leg
        sim.setJointTargetPosition(int(joint1), 0)
        sim.setJointTargetPosition(int(joint2), 0)

    return ["start" for _ in legs]

def default_position(legs):
    (th1, th2) = conversion(position("sit"))

    for leg in legs:
        (joint1, joint2) = leg
        sim.setJointTargetPosition(int(joint1), th1)
        sim.setJointTargetPosition(int(joint2), th2)

    return ["sit" for _ in legs]
        

In [13]:
# Change state of legs

def change_state(legs, legs_n, old_state, new_state, steps=10, duration=1, inter_method="linear"):
    """
    Change state of legs
    """

    assert len(legs) == len(old_state) == len(new_state), "Legs, old_state and new_state must have the same length"
    #print(f"Changing state of legs {legs_n} from {old_state} to {new_state}")
    # Get old positions
    olds = position([old_state[i] for i in legs_n])

    # Get new positions
    news = position([new_state[i] for i in legs_n])

    # Get legs operated
    legs_used = [legs[n] for n in legs_n]

    # Set legs to new positions
    set_legs(legs_used, olds, news, steps=steps, duration=duration, inter_method=inter_method)


# Possible actions
def follow_order(legs, order, old_state, steps=10, duration=1, follow=True):
    """
    Follow order of legs
    """

    action, legs_action, inter_method = order
    
    leg_n = None
    if legs_action == 'A':
        legs_n = [0,1,2,3]
    else:
        if type(legs_action) is int:
            legs_n = [legs_action]
        else:
            legs_n = legs_action

    if inter_method == 'l':
        inter_method = 'linear'
    elif inter_method == 'p':
        inter_method = 'parabolic'

    new_state = None
    if legs_action == 'A':
        if action == 'f':      new_state = forwards(old_state)
        elif action == 'b':   new_state = backwards(old_state)
        elif action == 'u':     new_state = upwards(old_state)
        elif action == 'd':   new_state = downwards(old_state)
    else:
        if type(legs_action) is int:
            new_state = old_state.copy()
            if action == 'f':
                new_state[legs_action] = forwards(old_state[legs_action])
            elif action == 'b':
                new_state[legs_action] = backwards(old_state[legs_action])
            elif action == 'ff':
                new_state[legs_action] = forwards(forwards(old_state[legs_action]))
            elif action == 'bb':
                new_state[legs_action] = backwards(backwards(old_state[legs_action]))
        else:
            new_state = old_state.copy()
            for n in legs_n:
                new_state[n] = action


    # Change state of legs
    if follow: 
        change_state(legs, legs_n, old_state, new_state, steps=steps, duration=duration, inter_method=inter_method)

    return new_state

def follow_sequence(sequence, legs, cycles=1, steps=10, duration=1, follow=True, show=False):
    """
    Follow sequence of orders
    """
    if follow:
       state = default_position(legs)
    else:
        state = ["sit" for _ in legs]

    if show:
        print(f"Initial state: {state}")
        print("Position = ", [position(state[i]) for i in range(len(state))])
        print("Angles = ", [conversion(position(state[i]), angle_type='degree') for i in range(len(state))])




    for order in sequence["start"]:
        state = follow_order(legs, order, state, steps=steps, duration=duration, follow=follow)
        
        if show:
            print(f"Executing start order: {order}, new state: {state}")
            print("Position = ", [position(state[i]) for i in range(len(state))])
            print("Angles = ", [conversion(position(state[i]), angle_type='degree') for i in range(len(state))])



    for _ in range(cycles):
        for order in sequence["cycle"]:
            state = follow_order(legs, order, state, steps=steps, duration=duration, follow=follow)
            
            if show:
                print(f"Executing cycle order: {order}, new state: {state}")
                print("Position = ", [position(state[i]) for i in range(len(state))])
                print("Angles = ", [conversion(position(state[i]), angle_type='degree') for i in range(len(state))])



    for order in sequence["end"]:
        state = follow_order(legs, order, state, steps=steps, duration=duration, follow=follow)
        
        if show:
            print(f"Executing end order: {order}, new state: {state}")
            print("Position = ", [position(state[i]) for i in range(len(state))])
            print("Angles = ", [conversion(position(state[i]), angle_type='degree') for i in range(len(state))])


    if follow:
       state = default_position(legs)
    else:
        state = ["sit" for _ in legs]
        
    if show:
        print(f"Final state: {state}")
        print("Position = ", [position(state[i]) for i in range(len(state))])
        print("Angles = ", [conversion(position(state[i]), angle_type='degree') for i in range(len(state))])


In [12]:
init_sim(LEGS)
time.sleep(20)
follow_sequence(walk_states, LEGS, cycles=10, steps=5, duration=0.5, follow=True, show=False)
follow_sequence(rot_states, LEGS, cycles=2, steps=5, duration=0.5, follow=True, show=False)
follow_sequence(maneta_states, LEGS, cycles=2, steps=5, duration=0.5, follow=True, show=False)
follow_sequence(sit_states, LEGS, cycles=2, steps=5, duration=0.5, follow=True, show=False)
follow_sequence(walk_back_states, LEGS, cycles=10, steps=5, duration=0.5, follow=True, show=False)


Changing state of legs [0, 1, 2, 3] from ['sit', 'sit', 'sit', 'sit'] to ['center', 'center', 'center', 'center']
Changing state of legs [0] from ['center', 'center', 'center', 'center'] to ['front', 'center', 'center', 'center']
Changing state of legs [0, 1, 2, 3] from ['front', 'center', 'center', 'center'] to ['center', 'back', 'back', 'back']
Changing state of legs [3] from ['center', 'back', 'back', 'back'] to ['center', 'back', 'back', 'center']
Changing state of legs [1] from ['center', 'back', 'back', 'center'] to ['center', 'front', 'back', 'center']
Changing state of legs [0, 1, 2, 3] from ['center', 'front', 'back', 'center'] to ['back', 'center', 'long_back', 'back']
Changing state of legs [2] from ['back', 'center', 'long_back', 'back'] to ['back', 'center', 'center', 'back']
Changing state of legs [0] from ['back', 'center', 'center', 'back'] to ['front', 'center', 'center', 'back']
Changing state of legs [0, 1, 2, 3] from ['front', 'center', 'center', 'back'] to ['center